[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Control de acceso mediante usuario y contraseña.

## La extensión Flask-Security.

Flask cuenta con varias extensiones enfocadas a garantizar la seguridad de los servicios web. Una de ellas es [Flask-Security](https://pythonhosted.org/Flask-Security/), la cual a su vez hace uso de las extensiones y módulos:

* [Flask-Login](https://flask-login.readthedocs.io/en/latest/).
* [Flask-Mail](https://pythonhosted.org/Flask-Mail/).
* [Flask-Principal](https://pythonhosted.org/Flask-Principal/).
* [Flask-WTF](https://flask-wtf.readthedocs.io/en/stable/).
* [itsdangerous](https://pythonhosted.org/itsdangerous/).
* [passlib](https://passlib.readthedocs.io/en/stable/).

Es necesario contar con un sistema de generación de *hash* como [bcrypt](https://pypi.org/project/bcrypt).

Esta extensión controla el proceso de alta, registro e incluso recuperación por correo electrónico de usuarios, contraseñas y roles.

In [ ]:
!pip install flask-security flask-sqlalchemy bcrypt 

## Ejemplo de una aplicación básica con Flask-Security y el ORM de Flask-SQLAlchemy.

Esta es una versión modificada del ejemplo localizado en https://pythonhosted.org/Flask-Security/quickstart.html#sqlalchemy-application. 

El uso de Flask-Security es aún más extenso, pero para los fines de este curso sólo se explorarán los aspectos más básicos.

### Importación de los módulos y paquetes.

* La extensión cuenta con objetos capaces de conectarse al ORM de SQLAlchemy y crear tablas especializadas para gestión de usuarios y roles en la base de datos.
* El uso de *mixins* facilita la creación de clases que contengan métodos compatibles con la extensión.

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_security import Security, SQLAlchemyUserDatastore, \
    UserMixin, RoleMixin, login_required

### Creación y configuración de la aplicación de Flask.

Para poder utilizar Flask-Security es necesario incluir algunos parámetros de configuración de Flask-SQLAlchemy así como:
* *'SECRET_KEY'*, el cual es utilizado por Flask-WTF para saegurar las formas.
* *'SECURITY_PASSWORD_SALT'* el cual sirve para cifrar las contraseñas.

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secretísimo'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/usuarios.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECURITY_PASSWORD_SALT'] = 'ultra-secreto'
db = SQLAlchemy(app)

### Definición de las tablas de usuarios y roles.

Se crearán varias tablas y clases.


**Advertencia:** para evitar conflictos es necesario eliminar el archivo *usuarios.db* localizado en el directorio [*data*](data) en caso de que exista.

#### Creación de la tabla de roles con la clase *db.Table*. 

La clase *db.Table()* permite definir un objeto ligado a una tabla en la base de datos, ingresando objetos instanciados de *db.Column* como argumentos para el método *\_\_init_\_()* del objeto instanciado.

Cabe hacer notar que la tabla *roles* contiene las columnas *id_usuario* y *id_rol*, las cuales están ligadas a los atributos *usuario.id* y *rol.id* respectivamente.

La clase *db.ForeginKey* liga a un campo ligado a la columna de otra tabla.

In [ ]:
roles_usuarios = db.Table('roles',
        db.Column('id_usuario', db.Integer(), db.ForeignKey('usuario.id')),
        db.Column('id_rol', db.Integer(), db.ForeignKey('rol.id')))

#### Creación de las tablas *rol* y *usuario* ligadas a las clases *Rol* y *Usuario* respectivamente. 

Los mixins *RoleMixin* y *UserMixin* ya contienen atributos y métodos capaces de crear tablas sin necesidad de definirlas explícitamente y ligar los objetos instanciados a dichas tablas. Por ello se crearán las tablas *rol* y *usuario* sin necesidad de especificarlo.

El atributo *Usuario.roles* utiliza la función *db.relationship(*) para definir una relacion ente a la clase *Usuario* y la clase *Rol* que se refleja en el objeto *roles_usuarios*.

Es decir, un usuario puede tener múltiples roles y esta relación de roles es guardada en la tabla *roles*, ligada al objeto *roles_usuarios*.

In [ ]:
help(db.relationship)

In [ ]:
class Rol(db.Model, RoleMixin):
    id = db.Column(db.Integer(), primary_key=True)
    name = db.Column(db.String(80), unique=True)
    description = db.Column(db.String(255))

class Usuario(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    password = db.Column(db.String(255))
    active = db.Column(db.Boolean())
    roles = db.relationship('Rol', secondary=roles_usuarios,
                            backref=db.backref('usuarios', lazy='dynamic'))

### Creación de las tablas en la base de datos.

Se crean las tablas en la base de datos.

In [ ]:
db.create_all()

Se puede comprobar el contenido de la base de datos de la siguiente manera:

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///data/usuarios.db

In [ ]:
%sql select tbl_name from sqlite_master where type = 'table';

### Creación del objeto que gestiona las operaciones de usuarios y roles.

La extensión Flask Security permite crear un objeto capaz de utilizar las tablas recién creadas para las operaciones de gestión de usuarios y roles mediante la clase *flask_security.SQLAlchemyUserDatastore*.

In [ ]:
datastore_usuario = SQLAlchemyUserDatastore(db, Usuario, Rol)

### Implementación de las operaciones de control de acceso  de la aplicación.

La clase *Security* permite crear un objeto el cual realiza las operaciones de control de acceso de la aplicación.

In [ ]:
seguridad = Security(app, datastore_usuario)

### Creación de un usario nuevo.

In [ ]:
datastore_usuario.create_user(email='contacto@pythonista.io', password='Jupyter')
db.session.commit()

In [ ]:
%sql select * from usuario

### Función de vista que requiere control de acceso mediante usuario y contraseña.

La función *flask_security.login_required()* aplicada como un decorador sobre la función *home()* indica que es necesario registrarse para pdoer acceder.

In [ ]:
@app.route('/')
@login_required
def home():
    return 'Hola'

### Ejecución de la aplicación.

Se ejecutará un servidor en http://localhost:5000 que pedirá un usuario y contraseña válidos para poder acceder.

**Advertencia:** Una vez ejecutada la siguente celda es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>